In [18]:
import qsharp
import qsharp_widgets
import json
from diskcache import Cache
import re_utils

re_utils.DEBUG = True

cache = Cache("~/quant-arith-cache/re-multipliers")
qsharp.init(project_root="../lib/")

@cache.memoize()
def estimate_resources_in_place_adder_pareto(op, n):
    est = qsharp.estimate(f"EstimateUtils.RunBinaryOpInPlace({n},{op})", params={"estimateType": "frontier"})
    return est

# These all are in-place adders modulo 2^n.
ops_and_max_n = [
  ("Std.Arithmetic.RippleCarryCGIncByLE", "Gidney", 2**20),
  ("Std.Arithmetic.RippleCarryTTKIncByLE", "TTK", 2**20),
  ("QuantumArithmetic.CDKM2004.Add", "CDKM", 2**20),
  ("QuantumArithmetic.DKRS2004.Add", "DKRS", 2**20),
  ("QuantumArithmetic.JHHA2016.Add_Mod2N", "JHHA", 2**20),
  ("QuantumArithmetic.TR2013.Add", "TR", 2**20),
  ("Std.Arithmetic.FourierTDIncByLE", "QFT", 9742),
]

In [23]:
n = 2**11
results = {} 
i = 0 
for op, _, _ in ops_and_max_n:
    results[i] = estimate_resources_in_place_adder_pareto(op, n)
    i += 1

qsharp_widgets.EstimatesOverview(results, runNames=[name for _, name, _ in ops_and_max_n])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…

The test above was limited to `n=2**11` due to the QFT based adder. However, below is a plot of the results of `n=2**20` without the QFT adder.

In [22]:
n = 2**20
results = {}
i = 0
for op, _, max_n in ops_and_max_n:
    if n <= max_n:
        results[i] =  estimate_resources_in_place_adder_pareto(op, n)
        i += 1

qsharp_widgets.EstimatesOverview(results, runNames=[name for _, name, max_n in ops_and_max_n if max_n >= n])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…

Since TTK, CDKM, JHHA, and TR overlap, below is a plot of the pareto frontier with `n=2**3` in order to show the differences.

In [16]:
closer_results = {}
n = 2**3
closer_results[0] = estimate_resources_in_place_adder_pareto("Std.Arithmetic.RippleCarryTTKIncByLE", n)
closer_results[1] = estimate_resources_in_place_adder_pareto("QuantumArithmetic.CDKM2004.Add",n)
closer_results[2] = estimate_resources_in_place_adder_pareto("QuantumArithmetic.JHHA2016.Add_Mod2N", n)
closer_results[3] = estimate_resources_in_place_adder_pareto("QuantumArithmetic.TR2013.Add", n)
qsharp_widgets.EstimatesOverview(closer_results, runNames=["TTK", "CDKM", "JHHA", "TR"])

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…